In [1]:
import os
import librosa
import soundfile as sf
import pandas as pd
from tqdm import tqdm

In [2]:
df = pd.read_excel('/Users/borosabel/Documents/Uni/Thesis/PopMIR/Data/Excel/baseline_data_w_topics_w_features.xlsx', engine='openpyxl')

In [3]:
df

,Artist,Album,Song,Coast,Release Year,Tempo1,Tempo2,Duration (s),Sample Rate (Hz),Path,...,Delta_MFCC_Mean_18,Delta_MFCC_Std_18,Delta2_MFCC_Mean_18,Delta2_MFCC_Std_18,Raw_MFCC_Mean_19,Raw_MFCC_Std_19,Delta_MFCC_Mean_19,Delta_MFCC_Std_19,Delta2_MFCC_Mean_19,Delta2_MFCC_Std_19
0,Big L,Lifestylez Ov Da Poor and Dangerous,8 Iz Enuff.mp3,east_coast,1995,96.774194,48.000000,298.840000,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,...,-0.000021,1.358306,-0.000043,0.915235,-3.153454,7.475548,0.000035,1.343144,0.000203,0.937476
1,Big L,Lifestylez Ov Da Poor and Dangerous,Da Graveyard.mp3,east_coast,1995,93.750000,46.511628,323.760000,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,...,0.000362,1.417600,-0.000426,1.016737,-0.245936,8.659272,0.000176,1.617870,-0.000542,1.091836
2,Big L,Lifestylez Ov Da Poor and Dangerous,I Don't Understand It.mp3,east_coast,1995,93.750000,47.244094,260.226667,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,...,0.000007,1.223709,0.000095,0.773686,-0.547386,7.437192,-0.000105,1.368790,0.000157,0.859366
3,Big L,Lifestylez Ov Da Poor and Dangerous,"No Endz, No Skinz.mp3",east_coast,1995,100.000000,50.420168,208.733333,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,...,-0.000214,1.164347,0.000087,0.769571,-1.566297,6.675558,-0.000312,1.268046,0.000172,0.856217
4,Big L,Lifestylez Ov Da Poor and Dangerous,MVP.mp3,east_coast,1995,86.956522,43.478261,218.866667,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,...,-0.000068,1.271479,-0.000235,0.895271,-2.257458,8.397979,-0.000024,1.502904,-0.000187,0.987014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1359,Dr.Dre,The Chronic,Dr. Dre - The Day the Niggaz Took Over (feat. ...,west_coast,1992,93.750000,46.875000,273.206000,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,...,0.000300,1.415803,0.000014,0.965833,3.105422,8.107130,0.000300,1.398945,-0.000331,0.943893
1360,Dr.Dre,The Chronic,"Dr. Dre - Bitches Ain't Shit (feat. Jewell, Sn...",west_coast,1992,92.307692,46.153846,287.207625,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,...,-0.000098,1.338693,-0.000104,0.923382,3.585479,8.419474,-0.000094,1.344639,-0.000100,0.945971
1361,Dr.Dre,The Chronic,Dr. Dre - Stranded On Death Row (feat. Bushwic...,west_coast,1992,90.909091,45.801527,287.335333,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,...,-0.000192,1.221769,-0.000096,0.872956,-1.408088,7.179779,-0.000186,1.220260,-0.000089,0.895696
1362,Dr.Dre,The Chronic,Dr. Dre - Nuthin' but a ＂G＂ Thang (feat. Snoop...,west_coast,1992,95.238095,47.244094,238.677917,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,...,0.000687,1.498789,0.000286,0.962360,3.666350,8.740702,-0.000545,1.380348,0.000392,0.903614


In [5]:
print(df[df['Coast'] == 'east_coast'].shape)
print(df[df['Coast'] == 'west_coast'].shape)

(698, 390)
(666, 390)


In [13]:
df[df['Release Year'] == 1996][['Artist', 'Album']]

,Artist,Album
74,A Tribe Called Quest,"Beats, Rhymes & Life"
75,A Tribe Called Quest,"Beats, Rhymes & Life"
76,A Tribe Called Quest,"Beats, Rhymes & Life"
77,A Tribe Called Quest,"Beats, Rhymes & Life"
78,A Tribe Called Quest,"Beats, Rhymes & Life"
...,...,...
1348,Dr.Dre,The Aftermath
1349,Dr.Dre,The Aftermath
1350,Dr.Dre,The Aftermath
1351,Dr.Dre,The Aftermath


In [33]:
import os
import librosa
import soundfile as sf
import pandas as pd
from tqdm import tqdm

def process_music_files(df, output_folder, pickle_filename):
    """
    Processes music files to calculate onsets, resample audio, and save results.
    
    Parameters:
    - df: pandas DataFrame with a column 'path' containing paths to music files.
    - output_folder: string, path to the folder where resampled audio files will be saved.
    - pickle_filename: string, filename for the output pickle file.
    
    Returns:
    - df_result: pandas DataFrame with columns 'path', 'onsets', 'num_onsets'.
    """
    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Lists to store results
    onsets_list = []
    num_onsets_list = []
    resampled_paths = []

    # Iterate over each file in the dataframe
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        music_path = row['Path']

        try:
            # Load audio file with original sampling rate
            y, sr = librosa.load(music_path, sr=None)

            # Calculate onsets (times in seconds)
            onset_frames = librosa.onset.onset_detect(y=y, sr=sr)
            onset_times = librosa.frames_to_time(onset_frames, sr=sr)
            onsets_list.append(onset_times.tolist())
            num_onsets_list.append(len(onset_times))

            # Resample audio to 44,100 Hz
            if sr != 44100:
                y_resampled = librosa.resample(y, orig_sr=sr, target_sr=44100)
                sr_resampled = 44100
            else:
                y_resampled = y
                sr_resampled = sr

            # Save resampled audio to output folder
            base_name = os.path.basename(music_path)
            resampled_path = os.path.join(output_folder, base_name)
            sf.write(resampled_path, y_resampled, sr_resampled)
            resampled_paths.append(resampled_path)

        except Exception as e:
            print(f"Error processing {music_path}: {e}")
            onsets_list.append([])
            num_onsets_list.append(0)
            resampled_paths.append(None)

    # Create the result dataframe
    df_result = df.copy()
    df_result['onsets'] = onsets_list
    df_result['num_onsets'] = num_onsets_list

    # Save dataframe to pickle file
    df_result.to_pickle(pickle_filename)

    return df_result

In [34]:
df_result = process_music_files(df, output_folder='/Users/borosabel/Documents/Uni/Thesis/PopMusicInformationRetrieval/Data/music2', pickle_filename='pkl_data/dataset_with_onset.pkl')

100%|██████████| 1364/1364 [38:37<00:00,  1.70s/it] 


In [20]:
df = pd.read_csv('./Out_19.csv')

In [27]:
df_processed = df.iloc[:, [10, -1, -2]]

In [28]:
df_processed

,/Users/borosabel/Documents/Uni/Thesis/PopMIR/Data/Audio/east_coast/Big L/Lifestylez Ov Da Poor and Dangerous/8 Iz Enuff.mp3,1650,"[0.14933333333333335, 0.384, 1.1306666666666667, 1.2906666666666666, 1.3333333333333333, 1.9093333333333333, 2.336, 2.656, 2.72, 3.296, 4.437333333333333, 4.874666666666666, 5.152, 5.386666666666667, 5.493333333333333, 7.509333333333333, 8.522666666666666, 8.650666666666666, 8.693333333333333, 8.885333333333334, 9.141333333333334, 9.514666666666667, 9.728, 10.048, 10.144, 10.314666666666668, 10.464, 10.762666666666666, 11.093333333333334, 11.296, 11.402666666666667, 11.562666666666667, 11.594666666666667, 11.690666666666667, 12.021333333333333, 12.202666666666667, 12.309333333333333, 12.544, 12.650666666666666, 12.768, 12.874666666666666, 12.96, 13.173333333333334, 13.269333333333334, 13.386666666666667, 13.557333333333334, 13.781333333333333, 13.898666666666667, 14.101333333333333, 14.208, 14.325333333333333, 14.410666666666666, 14.517333333333333, 14.837333333333333, 15.04, 15.146666666666667, 15.402666666666667, 15.466666666666667, 15.605333333333334, 15.712, 15.776, 15.957333333333333, 16.096, 16.128, 16.405333333333335, 16.597333333333335, 16.64, 16.714666666666666, 17.013333333333332, 17.248, 17.312, 17.536, 17.653333333333332, 17.962666666666667, 18.272, 18.464, 18.549333333333333, 18.816, 18.848, 19.125333333333334, 19.232, 19.370666666666665, 19.509333333333334, 19.68, 19.872, 20.053333333333335, 20.106666666666666, 20.149333333333335, 20.352, 20.469333333333335, 20.64, 20.75733333333333, 21.088, 21.290666666666667, 21.397333333333332, 21.568, 21.706666666666667, 21.866666666666667, 22.016, 22.282666666666668, ...]"
0,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,1811,"[0.128, 2.2826666666666666, 2.421333333333333,..."
1,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,1261,"[0.13866666666666666, 0.7893333333333333, 1.10..."
2,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,1006,"[0.14933333333333335, 4.074666666666666, 4.917..."
3,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,867,"[0.13866666666666666, 0.23466666666666666, 0.5..."
4,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,1030,"[0.128, 0.33066666666666666, 0.458666666666666..."
...,...,...,...
1358,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,1327,"[0.07466666666666667, 0.16, 1.7813333333333334..."
1359,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,1322,"[0.256, 0.416, 0.448, 0.6186666666666667, 0.94..."
1360,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,1779,"[0.30933333333333335, 0.384, 1.792, 1.89866666..."
1361,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,999,"[0.07466666666666667, 0.224, 0.874666666666666..."
